<a href="https://colab.research.google.com/github/GuilhermeMoraesLS/Projeto_SE_Maxmalize/blob/main/prototipo_autoguard_deteccao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 🧠 Instalações necessárias ---
!pip install cmake
!pip install dlib-bin
!pip install git+https://github.com/ageitgey/face_recognition_models
!pip install face-recognition --no-deps

# --- 📦 Importações ---
import os
import numpy as np
import cv2
import face_recognition
from google.colab import files
from google.colab.patches import cv2_imshow

# --- 📁 Criar pasta 'database' e fazer upload das imagens ---
os.makedirs("database", exist_ok=True)
print("📂 Crie sua base de dados de rostos")

uploaded = files.upload()  # escolha suas imagens de referência
for filename in uploaded.keys():
    os.rename(filename, f"database/{filename}")
print("✅ Imagens adicionadas à pasta 'database'")

# --- 🔍 Função para carregar banco de rostos ---
def load_face_database(path="database"):
    known_encodings = []
    known_names = []
    for filename in os.listdir(path):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            img = face_recognition.load_image_file(os.path.join(path, filename))
            encodings = face_recognition.face_encodings(img)
            if len(encodings) > 0:
                known_encodings.append(encodings[0])
                known_names.append(os.path.splitext(filename)[0])
    return known_encodings, known_names

# --- 🤖 Função otimizada para reconhecimento ---
def recognize_faces_in_image(image_path, known_encodings, known_names, tolerance=0.6, model="cnn", scale=0.25):
    """
    Reconhece rostos com redimensionamento automático para acelerar.
    """
    img = face_recognition.load_image_file(image_path)

    # Reduz resolução para acelerar processamento
    if scale != 1.0:
        small_img = cv2.resize(img, (0, 0), fx=scale, fy=scale)
    else:
        small_img = img

    img_rgb = cv2.cvtColor(small_img, cv2.COLOR_BGR2RGB)

    # Detecta rostos
    face_locations = face_recognition.face_locations(img_rgb, model=model)
    face_encodings = face_recognition.face_encodings(img_rgb, face_locations)

    results = []
    for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
        # Calcula distância dos rostos conhecidos
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)

        if len(face_distances) > 0:
            best_match_index = np.argmin(face_distances)
            if face_distances[best_match_index] < tolerance:
                name = known_names[best_match_index]
            else:
                name = "Desconhecido"
        else:
            name = "Desconhecido"

        # Ajusta coordenadas de volta ao tamanho original
        top = int(top / scale)
        right = int(right / scale)
        bottom = int(bottom / scale)
        left = int(left / scale)

        results.append((name, (top, right, bottom, left)))

    # Desenha retângulos e nomes
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    for name, (top, right, bottom, left) in results:
        cv2.rectangle(img_bgr, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(img_bgr, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return img_bgr, results

# --- 🧠 Carrega base de dados ---
print("📥 Carregando rostos conhecidos...")
known_encodings, known_names = load_face_database("database")
print(f"✅ Banco de dados carregado: {len(known_names)} rostos conhecidos.")

# --- 📸 Upload da imagem de teste ---
print("\n📤 Faça upload da imagem que deseja testar:")
uploaded_test = files.upload()
image_to_test = list(uploaded_test.keys())[0]

# --- 🧠 Executa reconhecimento ---
print("\n🔎 Analisando imagem...")
image_with_boxes, results = recognize_faces_in_image(
    image_to_test,
    known_encodings,
    known_names,
    tolerance=0.6,
    model="cnn",
    scale=0.25  # redimensiona para acelerar
)

# --- 📊 Resultados ---
print("\n📊 Resultados do reconhecimento:")
if not results:
    print("❌ Nenhum rosto foi detectado.")
else:
    for name, location in results:
        print(f"- Rosto identificado como: {name}")

# --- 📷 Exibe imagem final ---
print("\n🖼️ Imagem com resultados:")
cv2_imshow(image_with_boxes)

# --- 💾 Salva resultado ---
cv2.imwrite("resultado.jpg", image_with_boxes)
print("\n✅ Resultado salvo como 'resultado.jpg'")
